In [2]:
# Code from 
import torch
import math

dtype = torch.float
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(-math.pi, math.pi, 2000, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For a third order polynomial, we need
# 4 weights: y = a + b x + c x^2 + d x^3
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.randn((), dtype=dtype, requires_grad=True)
b = torch.randn((), dtype=dtype, requires_grad=True)
c = torch.randn((), dtype=dtype, requires_grad=True)
d = torch.randn((), dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y using operations on Tensors.
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
    # the gradient of the loss with respect to a, b, c, d respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

/home/jessica/miniconda3/envs/torch-test/lib/python3.12/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


99 2050.810791015625
199 1369.9517822265625
299 916.5589599609375
399 614.503662109375
499 413.1788330078125
599 278.9266357421875
699 189.35536193847656
799 129.5626983642578
899 89.62590026855469
999 62.93535614013672
1099 45.08642578125
1199 33.14250564575195
1299 25.14455223083496
1399 19.785064697265625
1499 16.19097900390625
1599 13.77891731262207
1699 12.158859252929688
1799 11.069828987121582
1899 10.337150573730469
1999 9.843790054321289
Result: y = -0.019176112487912178 + 0.8824307322502136 x + 0.003308200975880027 x^2 + -0.09698455035686493 x^3


In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv("stock-data.csv")
df.info()
df.drop("Date", inplace=True, axis = 1)
df = df.replace(",","", regex = True)
df = df.astype(float)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    232 non-null    object
 1   Open    232 non-null    object
 2   High    232 non-null    object
 3   Low     232 non-null    object
 4   Close   232 non-null    object
dtypes: object(5)
memory usage: 9.2+ KB


In [9]:
features = df[["Open", "High", "Low"]]
features = features.values
target = df[["Close"]]

              Open         High          Low
count   232.000000   232.000000   232.000000
mean   4746.149655  4766.391207  4725.216595
std     344.244756   343.212711   343.746747
min    4139.390000  4156.700000  4103.780000
25%    4457.387500  4479.260000  4443.580000
50%    4708.965000  4728.035000  4696.080000
75%    5087.270000  5106.450000  5057.555000
max    5340.260000  5354.160000  5302.400000


In [10]:
#Ww want to normalize our data to between -1 and 1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
target['Close'] = scaler.fit_transform(target['Close'].values.reshape(-1,1))

/tmp/ipykernel_2988935/4108026271.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target['Close'] = scaler.fit_transform(target['Close'].values.reshape(-1,1))


In [ ]:
#we want to remove the decimal places that get added there incoorectly
features = torch.round(torch.tensor(features, dtype = torch.float64)*100)/100

#this is beacuse we want a 1D array for our answerers
target = torch.round(torch.tensor(target, dtype = torch.float64).view(-1,1)*100)/100

tensor([[5314.4800, 5354.1600, 5297.6400],
        [5278.2400, 5298.8000, 5257.6300],
        [5297.1500, 5302.1100, 5234.3200],
        [5243.2100, 5280.3300, 5191.6800],
        [5259.7700, 5260.2100, 5222.1000],
        [5278.7300, 5282.2700, 5262.7000],
        [5315.9100, 5315.9100, 5280.8900],
        [5281.4500, 5311.6500, 5278.3900],
        [5340.2600, 5341.8800, 5256.9300],
        [5319.2800, 5323.1800, 5286.0100],
        [5298.6900, 5324.3200, 5297.8700],
        [5305.3500, 5325.3200, 5302.4000],
        [5303.1000, 5305.4500, 5283.5900],
        [5310.0700, 5325.4900, 5296.1900],
        [5263.2600, 5311.7600, 5263.2600],
        [5221.1000, 5250.3700, 5217.9800],
        [5233.0800, 5237.2600, 5211.1600],
        [5225.4900, 5239.6600, 5209.6800],
        [5189.0300, 5215.3000, 5180.4100],
        [5168.9800, 5191.9500, 5165.8600],
        [5187.2000, 5200.2300, 5178.9600],
        [5142.4200, 5181.0000, 5142.4200],
        [5122.7800, 5139.1200, 5101.2200],
        [50

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class stockPredictor(nn.Module):
    def __init__(self):
        super(stockPredictor, self).__init__()
        #this data has 4 inputs than we want to go to 64 i dont not understand why we chose these numbers
        self.fc1 = nn.Linear(3,16)
        self.fc2 = nn.Linear(16,63)
        self.fc3 = nn.Linear(63,32)
        self.fc4 = nn.Linear(32,1)
        self.double()
    
    #goes through each layer of the neural netwrok and returns an answer depending
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = stockPredictor()

In [ ]:
#we need to have the trainning and test in order due to the time relation of the data
x_train, x_test = features[0:int(len(features)*0.67),:], features[int(len(features)*0.67):len(features),:]
y_train, y_test = target[0:int(len(target)*0.67)], target[int(len(target)*0.67):len(target)]
print(y_test)

tensor([[4217.0400],
        [4224.1600],
        [4278.0000],
        [4314.6000],
        [4373.2000],
        [4373.6300],
        [4327.7800],
        [4349.6100],
        [4376.9500],
        [4358.2400],
        [4335.6600],
        [4308.5000],
        [4258.1900],
        [4263.7500],
        [4229.4500],
        [4288.3900],
        [4288.0500],
        [4299.7000],
        [4274.5100],
        [4273.5300],
        [4337.4400],
        [4320.0600],
        [4330.0000],
        [4402.2000],
        [4443.9500],
        [4453.5300],
        [4450.3200],
        [4505.1000],
        [4467.4400],
        [4461.9000],
        [4487.4600],
        [4457.4900],
        [4451.1400],
        [4465.4800],
        [4496.8300],
        [4515.7700],
        [4507.6600],
        [4514.8700],
        [4497.6300],
        [4433.3100],
        [4405.7100],
        [4376.3100],
        [4436.0100],
        [4387.5500],
        [4399.7700],
        [4369.7100],
        [4370.3600],
        [4404

In [ ]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)

epochs = 500
losses = []
for epoch in range(epochs):

    y_pred = model.forward(x_train)

    outputs = model(features)
    loss = criterion(y_pred, y_train)

    losses.append(loss.detach().numpy())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch +1) %10 ==0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss}')

Epoch [10/500], Loss: 4433184.434816257
Epoch [20/500], Loss: 705121.850834409
Epoch [30/500], Loss: 428.4625111854679
Epoch [40/500], Loss: 37236.9358622485
Epoch [50/500], Loss: 55350.38470273139
Epoch [60/500], Loss: 10167.545554068507
Epoch [70/500], Loss: 1184.5324342595657
Epoch [80/500], Loss: 3123.832074842822
Epoch [90/500], Loss: 371.55372511392915
Epoch [100/500], Loss: 685.3670276714427
Epoch [110/500], Loss: 372.17395404863805
Epoch [120/500], Loss: 415.9698906736182
Epoch [130/500], Loss: 384.97125243232733
Epoch [140/500], Loss: 370.75050407523213
Epoch [150/500], Loss: 370.99271961958664
Epoch [160/500], Loss: 370.62306322932193
Epoch [170/500], Loss: 370.8126741684905
Epoch [180/500], Loss: 371.0786574344307
Epoch [190/500], Loss: 371.0890192269184
Epoch [200/500], Loss: 370.97980272112625
Epoch [210/500], Loss: 370.86417266046635
Epoch [220/500], Loss: 370.7705041196323
Epoch [230/500], Loss: 370.6954060968022
Epoch [240/500], Loss: 370.63114846266893
Epoch [250/500],

In [ ]:
with torch.no_grad():
    y_eval = model.forward(x_test)
    loss = criterion(y_eval,y_test)

In [ ]:
num_correct = 0
resid = []
with torch.no_grad():
    for i, data in enumerate(x_test):
        y_val = model.forward(data)
        resid.append(round(y_test[i].item(),2) - round(y_val.item(),2))
        print(f"Predicted: {round(y_val.item(),2)} Expected: {round(y_test[i].item(),2)} Difference: {round(y_test[i].item(),2) - round(y_val.item(),2)}")
        if round(y_val.item(),2) == round(y_test[i].item(),2):
            num_correct += 1
print(f"Correct: {num_correct}")
print(np.std(resid))


Predicted: 4222.74 Expected: 4217.04 Difference: -5.699999999999818
Predicted: 4260.6 Expected: 4224.16 Difference: -36.44000000000051
Predicted: 4313.96 Expected: 4278.0 Difference: -35.960000000000036
Predicted: 4344.94 Expected: 4314.6 Difference: -30.339999999999236
Predicted: 4362.7 Expected: 4373.2 Difference: 10.5
Predicted: 4359.2 Expected: 4373.63 Difference: 14.430000000000291
Predicted: 4353.4 Expected: 4327.78 Difference: -25.61999999999989
Predicted: 4367.18 Expected: 4349.61 Difference: -17.57000000000062
Predicted: 4365.76 Expected: 4376.95 Difference: 11.1899999999996
Predicted: 4358.42 Expected: 4358.24 Difference: -0.18000000000029104
Predicted: 4308.94 Expected: 4335.66 Difference: 26.720000000000255
Predicted: 4266.17 Expected: 4308.5 Difference: 42.32999999999993
Predicted: 4253.22 Expected: 4258.19 Difference: 4.969999999999345
Predicted: 4244.24 Expected: 4263.75 Difference: 19.51000000000022
Predicted: 4259.19 Expected: 4229.45 Difference: -29.73999999999978
Pre

In [ ]:
correct = 0
resids = []
with torch.no_grad():
    for i, data in enumerate(x_train):
        y_val = model.forward(data)
        print(f"Predicted: {round(y_val.item(),2)} Expected: {round(y_train[i].item(),2)} Difference: {round(y_train[i].item(),2) - round(y_val.item(),2)}")
        resids.append(round(y_train[i].item(),2) - round(y_val.item(),2))
        if round(y_val.item(),2) == round(y_train[i].item(),2):
            correct += 1
print(np.std(resid))

Predicted: 5325.88 Expected: 5354.03 Difference: 28.149999999999636
Predicted: 5280.99 Expected: 5291.34 Difference: 10.350000000000364
Predicted: 5281.29 Expected: 5283.4 Difference: 2.1099999999996726
Predicted: 5243.36 Expected: 5277.51 Difference: 34.150000000000546
Predicted: 5249.56 Expected: 5235.48 Difference: -14.080000000000837
Predicted: 5276.13 Expected: 5266.95 Difference: -9.180000000000291
Predicted: 5306.31 Expected: 5306.04 Difference: -0.27000000000043656
Predicted: 5293.19 Expected: 5304.72 Difference: 11.530000000000655
Predicted: 5317.03 Expected: 5267.84 Difference: -49.1899999999996
Predicted: 5311.73 Expected: 5307.01 Difference: -4.719999999999345
Predicted: 5309.29 Expected: 5321.41 Difference: 12.11999999999989
Predicted: 5313.09 Expected: 5308.13 Difference: -4.960000000000036
Predicted: 5299.0 Expected: 5303.27 Difference: 4.270000000000437
Predicted: 5312.79 Expected: 5297.1 Difference: -15.6899999999996
Predicted: 5283.11 Expected: 5308.15 Difference: 25.